In [1]:
import numpy as np
import math
import PIL
from matplotlib.pyplot import imshow

In [2]:
import cv2
cap = cv2.VideoCapture('video1.mp4')

In [3]:
cap

<VideoCapture 0x7f91db231830>

In [ ]:
img = PIL.Image.open('lena.png', 'r')
key_img = PIL.Image.open('key1.2.jpg', 'r')

In [ ]:
key_img

In [ ]:
img_arr = np.asarray(img)
img_arr.shape

In [ ]:
key_img_arr = np.asarray(key_img)

In [ ]:
pix_val = list(img.getdata())

In [ ]:
pix_val_key = list(key_img.getdata())

In [ ]:
def make_gcd_matrix(key_img):
    
    pix_val_key = list(key_img.getdata())
    
    rows = key_img.size[0]
    cols = key_img.size[1]
    
    gcd_matrix = []
    sum_mod = []
    temp_li = []
    for pix_val in pix_val_key:
        gcd1 = math.gcd(pix_val[0], pix_val[1])
        gcd2 = math.gcd(pix_val[1], pix_val[2])
        gcd3 = math.gcd(pix_val[2], pix_val[0])
        gcd = (gcd1, gcd2, gcd3)
        temp_li.append(gcd)
        if len(temp_li) % key_img.size[0] == 0:
            if len(temp_li) == 0:
                temp_li = []
            else:
                gcd_matrix.append(temp_li)
                temp_li = []
          
    for i in range(len(gcd_matrix)):
        temp_sum_mod1 = 0
        temp_sum_mod2 = 0
        temp_sum_mod3 = 0
        for j in range(len(gcd_matrix[i])):
            if i == j:
                continue
            temp_sum_mod1 += gcd_matrix[i][j][0]
            temp_sum_mod2 += gcd_matrix[i][j][1]
            temp_sum_mod3 += gcd_matrix[i][j][2]
            
        temp_sum_mod = [temp_sum_mod1 % 256, temp_sum_mod2 % 256, temp_sum_mod3 % 256]
        
        sum_mod.append(temp_sum_mod)
    return sum_mod

In [ ]:
def make_matrix_table(sum_mod):
    dict_matrix_table = {}
    temp_list = [[i,i,i] for i in range(256)]
    
    for i in range(256):
        for k in range(len(temp_list)):
            for j in range(3):
                temp_list[k][j] += sum_mod[i % len(sum_mod)][j]
                temp_list[k][j] = temp_list[k][j] % len(temp_list)
   
        dict_matrix_table[i] = {}
        l = 0
        for j in temp_list:
            dict_matrix_table[i][l] = j
            l += 1
            
    return dict_matrix_table

In [ ]:
def pad_image(img, key):
    
    img_dim = img.size
    key_dim = key.size
    print(img_dim)
    print(key_dim)
    img_arr = np.asarray(img)
    pad_size_width = img_dim[0] % key_dim[0]
    pad_size_height = img_dim[1] % key_dim[1]
    
    if pad_size_width != 0 or pad_size_height != 0:
        arr_width = np.zeros((img_dim[1], key_dim[0]-pad_size_width, 3), dtype = 'uint8')
        arr_height = np.zeros((key_dim[1] - pad_size_height, (img_dim[0] + key_dim[0] - pad_size_width), 3), dtype = 'uint8')
        padded_img = np.concatenate((img_arr, arr_width), axis = 1)
        padded_img = np.concatenate((padded_img, arr_height), axis = 0)      
        return padded_img
    else:
        return np.asarray(img)

In [ ]:
sum_mod = make_gcd_matrix(key_img)

In [ ]:
dict_matrix_table = make_matrix_table(sum_mod)

In [ ]:
padded_img = pad_image(img, key_img)

In [ ]:
def encrypt_img(padded_img, key):

    key_dim = key.size
    cipher_img = np.empty(padded_img.shape, dtype = 'uint8')
    
    for i in range(padded_img.shape[0]):
        for j in range(padded_img.shape[1]):
            dict_key = padded_img[i%key_dim[0],j%key_dim[1]]
            for k in range(3):
                cipher_img[i][j][k] = dict_matrix_table[dict_key[k]][padded_img[i,j,k]][k]
                
    return cipher_img

In [ ]:
def trans(sum_mod):
    trans_mod = []
    for i in range(len(sum_mod)):
        temp = []
        for j in range(len(sum_mod[i])):
            temp.append(sum_mod[i][j] % len(sum_mod))
        trans_mod.append(temp)
    dim1 = []
    dim2 = []
    dim3 = []
    for i in range(len(trans_mod)):
        if trans_mod[i][0] not in dim1:
            dim1.append(trans_mod[i][0])
        
        if trans_mod[i][0] in dim1:
            for j in range(trans_mod[i][0]+1, len(trans_mod)):
                if j not in dim1:
                    dim1.append(j)
                    break
                if j == len(trans_mod) -1:
                    for k in range(0, trans_mod[i][0]):
                        if k not in dim1:
                            dim1.append(k)
                            break
                            
        if trans_mod[i][1] not in dim2:
            dim2.append(trans_mod[i][1])
            
        if trans_mod[i][1] in dim1:
            for j in range(trans_mod[i][1]+1, len(trans_mod)):
                if j not in dim2:
                    dim2.append(j)
                    break
                if j == len(trans_mod) -1:
                    for k in range(0, trans_mod[i][1]):
                        if k not in dim2:
                            dim2.append(k)
                            break

        if trans_mod[i][2] not in dim3:
            dim3.append(trans_mod[i][2])
            
        if trans_mod[i][2] in dim3:
            for j in range(trans_mod[i][2]+1, len(trans_mod)):
                if j not in dim3:
                    dim3.append(j)
                    break
                if j == len(trans_mod) -1:
                    for k in range(0, trans_mod[i][2]):
                        if k not in dim3:
                            dim3.append(k)
                            break
                            
    return dim1, dim2, dim3

In [ ]:
dim1, dim2, dim3 = trans(sum_mod)

In [ ]:
def encrypt_trans(dim1, dim2, dim3, subs_cipher_img):
    
    cipher_img = np.empty(subs_cipher_img.shape, dtype = 'uint8')
    
    for i in range(subs_cipher_img.shape[0]):
        for j in range(len(dim1)):
            index = dim1[j] 
            k = 0
            while index + k * len(dim1) < cipher_img.shape[1]:
                cipher_img[i,j + k * len(dim1),0] = subs_cipher_img[i, index + len(dim1)*k, 0]
                k += 1
            
    for i in range(subs_cipher_img.shape[0]):
        for j in range(len(dim2)):
            index = dim2[j] 
            k = 0
            while index + k * len(dim2) < cipher_img.shape[1]:
                cipher_img[i,j + k * len(dim2),1] = subs_cipher_img[i, index + len(dim2)*k, 1]
                k += 1

    for i in range(subs_cipher_img.shape[0]):
        for j in range(len(dim3)):
            index = dim3[j] 
            k = 0
            while index + k * len(dim3) < cipher_img.shape[1]:
                cipher_img[i,j + k * len(dim3),2] = subs_cipher_img[i, index + len(dim3)*k, 2]
                k += 1
            
    return cipher_img

In [ ]:
def encrypt_trans1(dim1, dim2, dim3, subs_cipher_img):
    
    cipher_img = np.empty(subs_cipher_img.shape, dtype = 'uint8')
    
    for j in range(subs_cipher_img.shape[1]):
        for i in range(len(dim1)):
            index = dim1[i] 
            k = 0
            while index + k * len(dim1) < cipher_img.shape[0]:
                cipher_img[i + k * len(dim1),j,0] = subs_cipher_img[index + len(dim1)*k, j, 0]
                k += 1
            
    for j in range(subs_cipher_img.shape[1]):
        for i in range(len(dim2)):
            index = dim2[i] 
            k = 0
            while index + k * len(dim2) < cipher_img.shape[0]:
                cipher_img[i+ k * len(dim2),j,1] = subs_cipher_img[index + len(dim2)*k, j, 1]
                k += 1

    for j in range(subs_cipher_img.shape[1]):
        for i in range(len(dim3)):
            index = dim3[i] 
            k = 0
            while index + k * len(dim3) < cipher_img.shape[0]:
                cipher_img[i+ k * len(dim3),j,2] = subs_cipher_img[index + len(dim3)*k, j, 2]
                k += 1
            
    return cipher_img

In [ ]:
def decrypt_trans1(dim1, dim2, dim3, cipher_img):
        
    plain_img = np.empty(cipher_img.shape, dtype = 'uint8')
    
    for j in range(cipher_img.shape[1]):
        for i in range(len(dim1)):
#             current = cipher_img[i,j,0]
            index = dim1[i]
            k = 0
            while index + k * len(dim1) < cipher_img.shape[0]:
                plain_img[index + k * len(dim1),j, 0] = cipher_img[i + k * len(dim1),j,0]
                k += 1
        
    for j in range(cipher_img.shape[1]):
        for i in range(len(dim2)):
            index = dim2[i]
            k = 0
            while index + k * len(dim2) < cipher_img.shape[0]:
                plain_img[index + k * len(dim2),j, 1] = cipher_img[i + k * len(dim2),j,1]
                k += 1
                
    for j in range(cipher_img.shape[1]):
        for i in range(len(dim3)):
            index = dim3[i]
            k = 0
            while index + k * len(dim3) < cipher_img.shape[1]:
                plain_img[index + k * len(dim3),j, 2] = cipher_img[i + k * len(dim3),j,2]
                k += 1
            
    return plain_img

In [ ]:
def decrypt_trans(dim1, dim2, dim3, cipher_img):
        
    plain_img = np.empty(cipher_img.shape, dtype = 'uint8')
    
    for i in range(cipher_img.shape[0]):
        for j in range(len(dim1)):
#             current = cipher_img[i,j,0]
            index = dim1[j]
            k = 0
            while index + k * len(dim1) < cipher_img.shape[1]:
                plain_img[i,index + k * len(dim1), 0] = cipher_img[i,j + k * len(dim1),0]
                k += 1
        
    for i in range(cipher_img.shape[0]):
        for j in range(len(dim2)):
            index = dim2[j]
            k = 0
            while index + k * len(dim2) < cipher_img.shape[1]:
                plain_img[i,index + k * len(dim2), 1] = cipher_img[i,j + k * len(dim2),1]
                k += 1
                
    for i in range(cipher_img.shape[0]):
        for j in range(len(dim3)):
            index = dim3[j]
            k = 0
            while index + k * len(dim3) < cipher_img.shape[1]:
                plain_img[i,index + k * len(dim3), 2] = cipher_img[i,j + k * len(dim3),2]
                k += 1
            
    return plain_img

In [ ]:
def decrypt_subs(cipher_img, key_img):
    
    sum_mod = make_gcd_matrix(key_img)
    dict_matrix_table = make_matrix_table(sum_mod)

    key_dim = key_img.size
    plain_img = np.empty(padded_img.shape, dtype = 'uint8')
    
    for i in range(cipher_img.shape[0]):
        for j in range(cipher_img.shape[1]):
            dict_key = cipher_img[i%key_dim[0],j%key_dim[1]]
            for k in range(3):
#                 ind = dict_matrix_table[dict_key[k]]
                for l in range(len(dict_matrix_table[dict_key[k]])):
                    if dict_matrix_table[dict_key[k]][l][k] == cipher_img[i,j,k]:
                        plain_img[i,j,k] = l
                        break
#         print(i)
                        
    return plain_img